In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score


import re
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [2]:
#Create all tuples of parameters
from itertools import product #Returns the cartesian product of lists (same as nested for loops)
def All_params_grid(dico):
    #Create list of keys and a list of values
    keys = list(dico.keys())
    values = dico.values()
    #Create all possible combinations of parameters
    params = []    
    for value in product(*values):#Loop trough all the combination of values
        subParam =dict() #reconstruct the dictionary of parameters
        for i in range(len(keys)):           
            subParam[str(keys[i])] = value[i] #Attribute the respective value to the respective key
        params.append(subParam)
    return(params)

In [3]:
# Before loading the following dataset, be sure to run the preprocessing ipython

In [4]:
data = pd.read_csv("1_dataset_ML_OHE_ready.csv", index_col=0 )

In [5]:
# # #take a sample for testing the code
# import random
# indecies = random.sample(range(len(data)), 500)
# data = data.iloc[indecies]

In [6]:
X = data.drop('readmitted', axis=1)
Y = data['readmitted']

In [7]:
#Split the dataset into training testing
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, test_size=0.1, random_state=101)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=101)

In [8]:
#Scale the data
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
X_train_scaled = Scaler.fit_transform(X_train)
X_validation_scaled = Scaler.transform(X_validation)
X_test_scaled = Scaler.transform(X_test)


In [9]:
# Example of a function to setup a deep neural network
#Build a wrapper function that returns the keras model

def build_keras_model(n_hidden_layers = 3,
                      activation_first= 'relu', 
                      activation_last = 'sigmoid', 
                      activation_middle_1 = 'relu',
                      activation_middle_2 = 'relu',
                      activation_before_decision_layers = 'relu',
                      n_nodes_first_layer = 128,  
                      n_nodes_deep_layers = 256, 
                      n_nodes_before_decision_layers = 128,
                      optimizer="adam"):
    
    model = tf.keras.Sequential()

    model.add(layers.Dense(n_nodes_first_layer, activation=activation_first))
    for i in range(n_hidden_layers):
        # Adds a densely-connected layer with 64 units to the model:
        if i%2 == 0:            
            model.add(layers.Dense(n_nodes_deep_layers, activation=activation_middle_1))            
        else:
            model.add(layers.Dense(n_nodes_deep_layers, activation=activation_middle_2))
            
        model.add(layers.Dropout(0.3))
            
        
    model.add(layers.Dense(n_nodes_before_decision_layers, activation=activation_before_decision_layers))
    model.add(layers.Dropout(0.3))  
        
    # Last layer that (activation depends on the number of categories in the outcome):
    model.add(layers.Dense(2, activation= activation_last ))
#     sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
#     model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    
    model.compile(optimizer=optimizer,
#               loss='categorical_crossentropy',
              loss='binary_crossentropy', 
              )
    
    return model


In [10]:
#Custom training method with cross validation
def trainKerasModel(params, x_train, y_train, x_validation, y_validation, n_epochs=100):
    '''This function is used to train the model without any ensemble method'''    
    #Create the model
    model = build_keras_model(**params)
    es = EarlyStopping(monitor='loss', mode='min', patience=5)
    model.fit(x_train, y_train.values, epochs=n_epochs, batch_size=32,
         callbacks = [es], verbose = 0)
    
    #Validation
    pred = model.predict_classes(x_validation)
    
    #build returned object
    
    obj = {
        'raw_model' : model,
        'auc_score_validation': roc_auc_score(pred, y_validation)
    }
    
    return (obj)

In [11]:
params = {
    'n_hidden_layers': [3, 5, 10, 15],
    'activation_first': ['relu', 'sigmoid'], 
    'activation_last': ['sigmoid'], 
    'activation_middle_1' : ['relu', 'sigmoid'],
    'activation_middle_2' : ['relu', 'sigmoid'],
    'activation_before_decision_layers' : ['relu', 'sigmoid'],
    'n_nodes_first_layer': [32, 64, 128],  
    'n_nodes_deep_layers': [128, 256], 
    'n_nodes_before_decision_layers': [128, 256],
    'optimizer':["adam", 'sgd']
}

In [12]:
params =  All_params_grid(params)

In [13]:
import random

In [14]:
#Random shuffle params
random.shuffle(params)

In [15]:
import warnings
warnings.filterwarnings(action="none")

AssertionError: invalid action: 'none'

In [ ]:
best_model = None

count = 0


for param in params: 
    
    count+=1

    print("Training model number {}/{}".format(count, len(params)))

    found = False
    
    try:
        model = trainKerasModel(param, X_train_scaled, y_train, X_validation_scaled, y_validation, n_epochs=1)
        
        if best_model is None:
            best_model = model
            found = True
        elif model['auc_score_validation'] > best_model['auc_score_validation']:
            best_model = model
            found = True
        
        
        if found:
            model['raw_model'].save("Deep_nn_best_model.h5")
            
    except:
        print("Problem with model number {} !".format(count))
        continue  

In [ ]:
#model = tf.keras.models.load_model('Deep_nn_best_model.h5')